In [ ]:
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
from pandas.core.common import random_state
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from tensorflow.python.ops.distributions.categorical import Categorical
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

### 데이터 읽어오기

In [ ]:
def extract(df):
    X = df.drop(columns=['MEDV'])
    Y = df['MEDV']
    return X, Y


def readCSV(fileName):
    df = pd.read_csv(fileName, dtype={'CHAS': 'bool', 'TAX': 'int'}).set_index('ID')
    return df


# 데이터 읽기
data = readCSV('data/train.csv')
test = readCSV('data/test.csv')

# 데이터 전처리
data = data[data['MEDV'] != 50.0]

# IQR 기반 이상치 제거 함수
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)  # 1사분위수
    Q3 = df[column].quantile(0.75)  # 3사분위수
    IQR = Q3 - Q1                   # IQR 계산
    
    # 이상치 범위 설정
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # 이상치를 제외한 데이터 필터링
    filtered_df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return filtered_df

# 효과 있는 것
# data = remove_outliers(data, 'CRIM')
# data = remove_outliers(data, 'RM')
# data = remove_outliers(data, 'B')
# data = remove_outliers(data, 'LSTAT')

# 무효과
# data = remove_outliers(data, 'INDUS')
# data = remove_outliers(data, 'NOX')
# data = remove_outliers(data, 'AGE')
# data = remove_outliers(data, 'RAD')
# data = remove_outliers(data, 'TAX')

# 역효과
# data = remove_outliers(data, 'ZN')
# data = remove_outliers(data, 'DIS')
# data = remove_outliers(data, 'PTRATIO')

# 데이터 전달
input_x, input_y = extract(data)
submit_x = test

input_x.info()

### 검증

In [ ]:
from sklearn.model_selection import cross_val_score

# 검증
model = LinearRegression()
rmse_scores = -cross_val_score(model, input_x, input_y, scoring='neg_root_mean_squared_error', cv=5)

# 평균 RMSE 출력
print(f"Average RMSE: {np.mean(rmse_scores):.4f}")

### 훈련 및 예측 파일 생성

In [ ]:
# 훈련
model.fit(input_x, input_y)

# 결과 파일 생성
submit_y = pd.DataFrame(model.predict(submit_x), index=submit_x.index, columns=['MEDV'])
submit_y.to_csv("submission.csv", index=True)